<a href="https://colab.research.google.com/github/Wickkey/Music-Generation-Analytics-Project/blob/main/Music_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json 
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import *
from music21 import *
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [2]:
data_directory = "./drive/My Drive/data/"
data_file = "Data_Tunes.txt"
charIndex_json = 'char_to_index.json'
BATCH_SIZE = 16
SEQ_LENGTH = 64


In [3]:
def preprocess(data):
  list1=list(data)
  list2=['\n','\n','\n']
  ignore=['X','T','M','S','K','P']
  i=0
  #to remove Part1:
  while(i<len(list1)):
    if(((list1[i] in ignore) and (list1[i+1]==":"))or list1[i]=='%' ):
      del list2[-1]
      while(list1[i]!='\n'):
        i=i+1
    list2.append(list1[i])
    i=i+1
  i=0
  #to append 'Z'(start token)
  preprocess_data=[]
  while(i<len(list2)):
    if(list2[i]=='\n'and list2[i+1]=='\n' and list2[i+2]=='\n'):
      preprocess_data.append('Z')
      i=i+3
    else:
      preprocess_data.append(list2[i])
      i=i+1
  return preprocess_data

In [4]:
def read_data(preprocess_data):
  char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(preprocess_data))))}

    
  with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
  index_to_char = {i: ch for (ch, i) in char_to_index.items()}
  num_unique_chars = len(char_to_index)
  all_characters_as_indices = np.asarray([char_to_index[c] for c in preprocess_data], dtype = np.int32)
  return all_characters_as_indices,num_unique_chars

In [5]:
def input_output(all_chars_as_indices,num_unique_chars):
    total_length = all_chars_as_indices.shape[0]
    num_examples=int(total_length/SEQ_LENGTH)
    X=np.zeros((num_examples,SEQ_LENGTH))
    Y=np.zeros((num_examples,SEQ_LENGTH,num_unique_chars))
    for i in range(num_examples):
      for j in range(SEQ_LENGTH):
        X[i,j]=all_chars_as_indices[i*SEQ_LENGTH+j]
        Y[i,j,all_chars_as_indices[i*SEQ_LENGTH+j+1]]=1
    return X,Y

In [6]:
def build_model( seq_length, num_unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = num_unique_chars, output_dim = 512, input_shape = (seq_length,))) 
    
    model.add(LSTM(256, return_sequences = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(256, return_sequences = True))
    model.add(Dropout(0.2))
    
    
    model.add(TimeDistributed(Dense(num_unique_chars)))

    model.add(Activation("softmax"))
    
    return model

In [7]:
def make_model(num_unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = num_unique_chars, output_dim = 512, batch_input_shape = (1, 1))) 
  
# stateful: If True, the last state for each sample at index i in a batch will be used 
# as initial state for the sample of index i in the following batch.
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256,return_sequences=True, stateful = True)) 
    model.add(Dropout(0.2))
    
    model.add((Dense(num_unique_chars)))
    model.add(Activation("softmax"))
    
    return model

In [8]:
def generate_sequence(gen_seq_length):
    with open(os.path.join(data_directory, charIndex_json)) as f:
        char_to_index = json.load(f)
    index_to_char = {i:ch for ch, i in char_to_index.items()}
    num_unique_chars = len(index_to_char)
    
    model = make_model(num_unique_chars)
    model.load_weights("./drive/My Drive/weights.80.hdf5")
     
    sequence_index = [char_to_index['A'],char_to_index['E'],char_to_index['D'],char_to_index['#'],char_to_index['C'],char_to_index['B'],]

    for _ in range(gen_seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        
        predicted_probs = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(num_unique_chars), size = 1, p = predicted_probs)
        
        
        sequence_index.append(sample[0])
    
        
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    seq='M:4/8\n'+str(seq)
    return seq

In [9]:
def convert_to_midi(abc):
    c = converter.subConverters.ConverterABC()
    c.registerOutputExtensions = ("midi", )
    c.parseData(abc)
    s = c.stream
    s.write('midi', fp='demos1.mid')

In [10]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
preprocess_data=preprocess(data)
print(set(preprocess_data))
all_characters_as_indices,num_unique_chars=read_data(preprocess_data)
X,Y=input_output(all_characters_as_indices,num_unique_chars)
print("length of preprocess_data-{}".format(len(preprocess_data)))
print("vocab_size={}".format(num_unique_chars))
print("all_characters={}".format(all_characters_as_indices))
print("length of all_characters-{}".format(len(all_characters_as_indices)))
print("shape of X={}".format(X.shape))
print("shape of Y={}".format(Y.shape))

{'Z', 'f', '(', '^', 't', ',', 'b', 'r', '|', ':', 'C', 'z', 'L', 'B', '!', ']', 'D', 'c', '\n', 'u', 'o', '[', '=', '3', 'a', 's', '4', ')', '8', '"', 'p', '-', '~', '2', 'l', '+', 'd', '1', 'F', 'i', '6', 'R', '/', 'e', 'E', 'g', '\\', 'G', '_', '7', 'm', 'V', '#', "'", 'A', 'H', 'n', '9', ' '}
length of preprocess_data-116963
vocab_size=59
all_characters=[33 44 57 ... 15 20 57]
length of all_characters-116963
shape of X=(1827, 64)
shape of Y=(1827, 64, 59)


In [ ]:
model=build_model(SEQ_LENGTH,num_unique_chars)
model.summary()
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
checkpoint=ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5',monitor='loss',save_best_only=True,save_weights_only=True,period=1)
model.fit(X,Y,batch_size=16,epochs=80,callbacks=[checkpoint])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 64, 512)           30208     
_________________________________________________________________
lstm (LSTM)                  (None, 64, 256)           787456    
_________________________________________________________________
dropout (Dropout)            (None, 64, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64, 256)           525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64, 256)           525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 256)           0

In [ ]:
music = generate_sequence(192)
print("\nMUSIC SEQUENCE GENERATED: \n{}".format(music))
convert_to_midi(music)
